# Vidu Q2 Text-to-Video API 使用指南

本 Notebook 演示如何通过 **七牛 AIToken** 平台调用 **Vidu Q2 (Text-to-Video)** 模型 API。

## 模型简介

Vidu Q2 是一款强大的文本生成视频模型，支持以下能力：

- **文本生成视频**：根据文本提示词自动生成高质量视频
- **多种宽高比**：支持 16:9、9:16、3:4、4:3、1:1 等多种比例
- **运动幅度控制**：支持 auto、small、medium、large 四种运动幅度
- **背景音乐**：可选自动添加匹配的背景音乐

**API 端点**：`fal-ai/vidu/q2/text-to-video`

## 1. 环境准备

首先安装 `fal-client` 依赖包，并设置 API Key 和七牛 AIToken 队列服务地址。

In [ ]:
# 安装 fal-client
%pip install fal-client -q

In [ ]:
import os

# 设置七牛 AIToken 队列服务地址
os.environ["FAL_QUEUE_RUN_HOST"] = "api.qnaigc.com/queue"

# 设置 FAL_KEY 环境变量
# 你也可以在终端中通过 export FAL_KEY="xxx" 来设置
os.environ["FAL_KEY"] = os.environ.get("QINIU_API_KEY") or (_ for _ in ()).throw(ValueError("环境变量 QINIU_API_KEY 未设置"))

In [ ]:
import fal_client
from IPython.display import Video, display

## 2. 基础用法 — 队列调用

七牛 AIToken 平台使用队列模式调用 API：先提交请求（`submit`），然后轮询状态（`status`），最后获取结果（`result`）。

下面先封装一个通用的调用辅助函数，后续示例都会复用。

In [ ]:
import time

def run_fal_task(endpoint, arguments, poll_interval=5):
    """提交任务并轮询等待结果返回"""
    # 步骤 1：提交请求
    handler = fal_client.submit(endpoint, arguments=arguments)
    request_id = handler.request_id
    print(f"请求已提交，request_id: {request_id}")

    # 步骤 2：轮询任务状态
    while True:
        status = fal_client.status(endpoint, request_id, with_logs=True)
        print(f"当前状态: {type(status).__name__}")

        if isinstance(status, fal_client.Completed):
            print("任务完成!")
            break

        if hasattr(status, "logs") and status.logs:
            for log in status.logs:
                print(log["message"])

        time.sleep(poll_interval)

    # 步骤 3：获取结果
    return fal_client.result(endpoint, request_id)


# 基础调用示例
result = run_fal_task(
    "fal-ai/vidu/q2/text-to-video",
    arguments={
        "prompt": "一只可爱的橘猫在阳光下追逐蝴蝶，慢镜头，电影质感，温暖的光线",
        "aspect_ratio": "16:9",
        "bgm": False,
    },
)

# 打印返回结果
print(result)

In [ ]:
# 展示生成的视频
video_url = result["video"]["url"]
print(f"视频 URL: {video_url}")
display(Video(url=video_url, width=640))

## 3. 参数说明与高级用法

### 可配置参数一览

| 参数 | 类型 | 默认值 | 说明 |
|------|------|--------|------|
| `prompt` | string | *必填* | 文本提示词，最多 2000 字符 |
| `aspect_ratio` | string | *必填* | 宽高比：`16:9`、`9:16`、`3:4`、`4:3`、`1:1`（其中 `3:4`、`4:3` 仅支持 Q2 模型） |
| `bgm` | boolean | *必填* | 是否添加背景音乐，`true` 自动匹配 BGM，`false` 不添加 |
| `seed` | integer | 随机 | 随机种子，默认不传或传 0 使用随机数 |
| `duration` | integer | `5` | 视频时长（秒），可选：`1-10` |
| `resolution` | string | `"1080p"` | 分辨率，可选：`1080p` |
| `movement_amplitude` | string | `"auto"` | 运动幅度：`auto`、`small`、`medium`、`large` |

In [ ]:
# 高级用法：使用更多参数
result_advanced = run_fal_task(
    "fal-ai/vidu/q2/text-to-video",
    arguments={
        "prompt": "赛博朋克风格的未来城市夜景，霓虹灯闪烁，飞行汽车穿梭在摩天大楼之间，细雨蒙蒙，电影级画质",
        "aspect_ratio": "16:9",
        "bgm": True,                    # 添加背景音乐
        "seed": 42,                      # 固定随机种子，便于复现
        "duration": 5,                   # 视频时长 5 秒
        "resolution": "1080p",           # 1080p 分辨率
        "movement_amplitude": "medium",  # 中等运动幅度
    },
)

# 展示结果
video_url = result_advanced["video"]["url"]
print(f"视频 URL: {video_url}")
display(Video(url=video_url, width=640))

## 4. 队列模式 — 手动分步调用

上面的 `run_fal_task` 封装了完整的队列流程。如果你需要更精细的控制（例如在提交后做其他事情，稍后再取结果），可以手动分步调用。

In [ ]:
# 步骤 1：提交请求
handler = fal_client.submit(
    "fal-ai/vidu/q2/text-to-video",
    arguments={
        "prompt": "一片宁静的湖面上，晨雾缓缓散去，远处山峦若隐若现，一只白鹤展翅飞过水面",
        "aspect_ratio": "16:9",
        "bgm": False,
    },
)

# 获取请求 ID，用于后续查询
request_id = handler.request_id
print(f"请求已提交，request_id: {request_id}")

In [ ]:
# 步骤 2：查询任务状态
import time

while True:
    status = fal_client.status(
        "fal-ai/vidu/q2/text-to-video",
        request_id,
        with_logs=True,
    )
    print(f"当前状态: {type(status).__name__}")

    if isinstance(status, fal_client.Completed):
        print("任务完成!")
        break

    # 每 5 秒查询一次
    time.sleep(5)

In [ ]:
# 步骤 3：获取结果
result_async = fal_client.result(
    "fal-ai/vidu/q2/text-to-video",
    request_id,
)

# 展示生成的视频
video_url = result_async["video"]["url"]
print(f"视频 URL: {video_url}")
display(Video(url=video_url, width=640))

## 5. 竖屏视频示例

使用 `9:16` 比例生成适合短视频平台的竖屏内容。

In [ ]:
# 竖屏视频示例
result_portrait = run_fal_task(
    "fal-ai/vidu/q2/text-to-video",
    arguments={
        "prompt": "一位身穿汉服的女子在樱花树下翩翩起舞，花瓣飘落，柔和的逆光，唯美浪漫的氛围",
        "aspect_ratio": "9:16",
        "bgm": True,
        "movement_amplitude": "large",
    },
)

# 展示结果
video_url = result_portrait["video"]["url"]
print(f"视频 URL: {video_url}")
display(Video(url=video_url, width=360))

## 6. Schema 参考

### Input Schema

```json
{
  "prompt": "string (必填, 最多 2000 字符)",
  "aspect_ratio": "16:9 | 9:16 | 3:4 | 4:3 | 1:1 (必填)",
  "bgm": "boolean (必填)",
  "seed": "integer (可选, 默认随机)",
  "duration": "integer (可选, 默认 5)",
  "resolution": "1080p (可选, 默认 1080p)",
  "movement_amplitude": "auto | small | medium | large (可选, 默认 auto)",
}
```

### Output Schema (队列提交响应)

```json
{
  "status": "IN_QUEUE",
  "request_id": "string (任务唯一 ID)",
  "response_url": "string (查询任务结果的 URL)",
  "status_url": "string (查询任务状态的 URL)",
  "cancel_url": "string (取消任务的 URL, 暂未支持)"
}
```

### 更多资源

- [七牛 AIToken 文档](https://developer.qiniu.com/aitokenapi)
- [fal-client Python 文档](https://docs.fal.ai/reference/client-libraries/python/fal_client)
- [七牛 API 调用示例文档](https://apidocs.qnaigc.com/417911128e0)